In [7]:
import pickle
import numpy as np
import pandas as pd

In [35]:
with open("raw_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [36]:
def build_comparison_df(sensor_data, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_data_len = sensor_data[f"mat_{
            mat}"][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)

    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_dates = sensor_data[f"mat_{
            mat}"][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates

    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]
                        ).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds

    df = pd.DataFrame(heater_dict)
    return df

In [9]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [39]:
MATRIX = 0

In [40]:
for i in range(8):
    df = build_comparison_df(sensor_data, MATRIX, i)
    df.to_excel(f"mat_{MATRIX}_s{i}_diff.xlsx")

In [29]:
sensor_data[f"mat_{MATRIX}"][6][6].iloc[729:732]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
87525,6,355877689,19634632,1728742589,39.307915,920.931213,24.914812,6222560.0,6,1,1,0,0,2024-10-12 17:16:29
87765,6,355877689,19688361,1728742643,39.460129,920.949646,24.871605,6215478.0,6,1,1,0,0,2024-10-12 17:17:23
87877,6,355877689,19715221,1728742670,39.412716,920.958740,24.851255,6240335.0,6,1,1,0,0,2024-10-12 17:17:50


In [31]:
sensor = 6
heater = 6
after = 729
df_copy = impute_data_mean(sensor_data[f"mat_{MATRIX}"][sensor][heater], after)
df_copy.iloc[after:after+4]
# sensor_data[f"mat_{MATRIX}"][sensor][heater] = df_copy.copy()

In [34]:
with open("imputed_sensor_data.pkl", "wb") as f:
    pickle.dump(sensor_data, f)